# Setup

In [1]:
import os
import openai
 
openai.api_key = os.environ["OPENAI_API_KEY"]

def llm(prompt, stop=["\n"]):
    response = openai.Completion.create(
      model="text-davinci-002",
      prompt=prompt,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

In [2]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment

WEBSHOP_URL = "http://3.83.245.205:3000"
ACTION_TO_TEMPLATE = {
    'Description': 'description_page.html',
    'Features': 'features_page.html',
    'Reviews': 'review_page.html',
    'Attributes': 'attributes_page.html',
}

def clean_str(p):
  return p.encode().decode("unicode-escape").encode("latin1").decode("utf-8")


def tag_visible(element):
    ignore = {'style', 'script', 'head', 'title', 'meta', '[document]'}
    return (
        element.parent.name not in ignore and not isinstance(element, Comment)
    )


def webshop_text(session, page_type, query_string='', page_num=1, asin='', options={}, subpage='', **kwargs):
    if page_type == 'init':
      url = (
          f'{WEBSHOP_URL}/{session}'
      )
    if page_type == 'search':
      url = (
          f'{WEBSHOP_URL}/search_results/{session}/'
          f'{query_string}/{page_num}'
      )
    elif page_type == 'item':
      url = (
          f'{WEBSHOP_URL}/item_page/{session}/'
          f'{asin}/{query_string}/{page_num}/{options}'
      )
    elif page_type == 'item_sub':
      url = (
          f'{WEBSHOP_URL}/item_sub_page/{session}/'
          f'{asin}/{query_string}/{page_num}/{subpage}/{options}'
      )
    elif page_type == 'end':
      url = (
          f'{WEBSHOP_URL}/done/{session}/'
          f'{asin}/{options}'
      )
    # print(url)
    html = requests.get(url).text
    html_obj = BeautifulSoup(html, 'html.parser')
    texts = html_obj.findAll(text=True)
    visible_texts = list(filter(tag_visible, texts))
    # visible_texts = [str(text).strip().strip('\\n') for text in visible_texts]
    # if page_type == 'end': import pdb; pdb.set_trace()
    if False:
        # For `simple` mode, return just [SEP] separators
        return ' [SEP] '.join(t.strip() for t in visible_texts if t != '\n')
    else:
        # Otherwise, return an observation with tags mapped to specific, unique separators
        observation = ''
        option_type = ''
        options = {}
        asins = []
        cnt = 0
        prod_cnt = 0
        just_prod = 0
        for t in visible_texts:
            if t == '\n': continue
            if t.replace('\n', '').replace('\\n', '').replace(' ', '') == '': continue
            # if t.startswith('Instruction:') and page_type != 'init': continue
            # print(t.parent.name, t)
            if t.parent.name == 'button':  # button
                processed_t = f'\n[{t}] '
            elif t.parent.name == 'label':  # options
                if f"'{t}'" in url:
                    processed_t = f'[[{t}]]'
                    # observation = f'You have clicked {t}.\n' + observation
                else:
                    processed_t = f'[{t}]'
                options[str(t)] = option_type
                # options[option_type] = options.get(option_type, []) + [str(t)]
            elif t.parent.get('class') == ["product-link"]: # product asins
                processed_t = f'\n[{t}] '
                if prod_cnt >= 3:
                  processed_t = ''
                prod_cnt += 1
                asins.append(str(t))
                just_prod = 0
            else: # regular, unclickable text
                processed_t =  '\n' + str(t) + ' '
                if cnt < 2 and page_type != 'init': processed_t = ''
                if just_prod <= 2 and prod_cnt >= 4: processed_t = ''
                option_type = str(t)
                cnt += 1
            just_prod += 1
            observation += processed_t
        info = {}
        if options:
          info['option_types'] = options
        if asins:
          info['asins'] = asins
        if 'Your score (min 0.0, max 1.0)' in visible_texts:
          idx = visible_texts.index('Your score (min 0.0, max 1.0)')
          info['reward'] = float(visible_texts[idx + 1])
          observation = 'Your score (min 0.0, max 1.0): ' + (visible_texts[idx + 1])
        return clean_str(observation), info

class webshopEnv:
  def __init__(self):
    self.sessions = {}
  
  def step(self, session, action):
    done = False
    observation_ = None
    if action == 'reset':
      self.sessions[session] = {'session': session, 'page_type': 'init'}
    elif action.startswith('think['):
      observation = 'OK.'
    elif action.startswith('search['):
      assert self.sessions[session]['page_type'] == 'init'
      query = action[7:-1]
      self.sessions[session] = {'session': session, 'page_type': 'search',
                                'query_string': query, 'page_num': 1}
    elif action.startswith('click['):
      button = action[6:-1]
      if button == 'Buy Now':
        assert self.sessions[session]['page_type'] == 'item'
        self.sessions[session]['page_type'] = 'end'
        done = True
      elif button == 'Back to Search':
        assert self.sessions[session]['page_type'] in ['search', 'item_sub', 'item']
        self.sessions[session] = {'session': session, 'page_type': 'init'}
      elif button == 'Next >':
        assert False # ad hoc page limitation
        assert self.sessions[session]['page_type'] == 'search'
        self.sessions[session]['page_num'] += 1
      elif button == '< Prev':
        assert self.sessions[session]['page_type'] in ['search', 'item_sub', 'item']
        if self.sessions[session]['page_type'] == 'search':
          assert False
          self.sessions[session]['page_num'] -= 1
        elif self.sessions[session]['page_type'] == 'item_sub':
          self.sessions[session]['page_type'] = 'item'
        elif self.sessions[session]['page_type'] == 'item':
          self.sessions[session]['page_type'] = 'search'
          self.sessions[session]['options'] = {}
      elif button in ACTION_TO_TEMPLATE:
        assert self.sessions[session]['page_type'] == 'item'
        self.sessions[session]['page_type'] = 'item_sub'
        self.sessions[session]['subpage'] = button
      else:
        if self.sessions[session]['page_type'] == 'search':
          assert button in self.sessions[session].get('asins', [])  # must be asins
          self.sessions[session]['page_type'] = 'item'
          self.sessions[session]['asin'] = button
        elif self.sessions[session]['page_type'] == 'item':
          assert 'option_types' in self.sessions[session]
          assert button in self.sessions[session]['option_types'], (button, self.sessions[session]['option_types'])  # must be options
          option_type = self.sessions[session]['option_types'][button]
          if not 'options' in self.sessions[session]:
            self.sessions[session]['options'] = {}
          self.sessions[session]['options'][option_type] = button
          observation_ = f'You have clicked {button}.'
    else:
      assert False
    observation, info = webshop_text(**self.sessions[session])
    if observation_:
      observation = observation_
    self.sessions[session].update(info)
    reward = info.get('reward', 0.0)
    return observation, reward, done
env = webshopEnv()



# ReAct

In [3]:
# trivial search & item, choose option
prompt1 = """Webshop 
Instruction:  
i would like a 3 ounce bottle of bright citrus deodorant for sensitive skin, and price lower than 50.00 dollars 
[Search]  

Action: search[3 ounce bright citrus deodorant sensitive skin]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B078GWRC1J] 
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce 
$10.99 
[B078GTKVXY] 
Ginger Fresh Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce 
$10.99 
[B08KBVJ4XN] 
Barrel and Oak - Aluminum-Free Deodorant, Deodorant for Men, Essential Oil-Based Scent, 24-Hour Odor Protection, Cedar & Patchouli Blend, Gentle on Sensitive Skin (Mountain Sage, 2.7 oz, 2-Pack) 
$15.95  

Action: think[B078GWRC1J and B078GTKVXY are bright citrus deodorant less then 50 dollars. I can check B078GWRC1J first.]
Observation: OK.

Action: click[B078GWRC1J]
Observation: 
[Back to Search] 
[< Prev] 
scent [assorted scents][bright citrus][calming lavender][ginger fresh][simply non-scents]
size [travel set (4-pack)][3 ounce (pack of 1)][3-ounce (2-pack)]
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce 
Price: $10.99 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now]  

Action: think[For 3 ounce bottle of bright citrus deodorant for sensitive skin, the item has options 'bright citrus' and '3 ounce (pack of 1)' and seems good to buy.]
Observation: OK.

Action: click[bright citrus]
Observation: You have clicked bright citrus. 

Action: click[3 ounce (pack of 1)]
Observation: You have clicked 3 ounce (pack of 1). 

Action: click[Buy Now]
"""

In [4]:
# trivial search & item, choose option
prompt1_actonly = """Webshop 
Instruction:  
i would like a 3 ounce bottle of bright citrus deodorant for sensitive skin, and price lower than 50.00 dollars 
[Search]  

Action: search[3 ounce bright citrus deodorant sensitive skin]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B078GWRC1J] 
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce 
$10.99 
[B078GTKVXY] 
Ginger Fresh Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce 
$10.99 
[B08KBVJ4XN] 
Barrel and Oak - Aluminum-Free Deodorant, Deodorant for Men, Essential Oil-Based Scent, 24-Hour Odor Protection, Cedar & Patchouli Blend, Gentle on Sensitive Skin (Mountain Sage, 2.7 oz, 2-Pack) 
$15.95  

Action: click[B078GWRC1J]
Observation: 
[Back to Search] 
[< Prev] 
scent [assorted scents][bright citrus][calming lavender][ginger fresh][simply non-scents]
size [travel set (4-pack)][3 ounce (pack of 1)][3-ounce (2-pack)]
Bright Citrus Deodorant by Earth Mama | Natural and Safe for Sensitive Skin, Pregnancy and Breastfeeding, Contains Organic Calendula 3-Ounce 
Price: $10.99 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now]  

Action: click[bright citrus]
Observation: You have clicked bright citrus. 

Action: click[3 ounce (pack of 1)]
Observation: You have clicked 3 ounce (pack of 1). 

Action: click[Buy Now]
"""

In [5]:
def webshop_run(idx, prompt, to_print=True):
  action = 'reset'
  init_prompt = prompt
  prompt = ''
  for i in range(15):
    try:
      res = env.step(idx, action)
      observation = res[0]
    except AssertionError:
      observation = 'Invalid action!'

    if action.startswith('think'):
      observation = 'OK.'


    if to_print:
      print(f'Action: {action}\nObservation: {observation}\n')
      sys.stdout.flush()
    if i:
      prompt += f' {action}\nObservation: {observation}\n\nAction:'
    else:
      prompt += f'{observation}\n\nAction:'
    
    if res[2]:  
      return res[1]

    action = llm(init_prompt + prompt[-(6400-len(init_prompt)):], stop=['\n']).lstrip(' ')

  return 0

def run_episodes(prompt, n=50):
  rs = []
  cnt = 0
  for i in range(n):
    print('-----------------')
    print(i)
    try:
      r = webshop_run(f'fixed_{i}', prompt, to_print=True)
    except AssertionError:
      r = 0
      cnt += 1
    rs.append(r)
    if (i+1) % 1 == 0:
      r, sr, fr = sum(rs) / len(rs), len([_ for _ in rs if _ == 1]) / len(rs), cnt / len(rs)
      print(i+1, r, sr, fr)
      print('-------------')
  r, sr, fr = sum(rs) / len(rs), len([_ for _ in rs if _ == 1]) / n, cnt / n
  print(r, sr, fr)
  return rs

In [6]:
res1 = run_episodes(prompt1, 500)

-----------------
0
Action: reset
Observation: 
Webshop 
Instruction:  
i need a long clip-in hair extension which is natural looking, and price lower than 20.00 dollars 
[Search] 

Action: search[long clip-in hair extension natural looking]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B09QQLDJ93] 
Lroveb Fashion Long Curly Wig Head Dyed Black Gradient Golden Long Straight Wig African Long Curly Hair Wigs Ladies Gradient Color Chemical Fiber Silk Corn Perm Wig for Women 
$34.89 
[B09QQKPZJR] 
Lroveb Beanie Hat Knit with Long Wavy Curly Hair Extensions Wig Winter Warm Fashion Long Curly Hair Hat Wig Wool Curl Corn Perm Curl Knitted Cap Attached Synthetic Hairpiece 
$36.06 
[B08H5DCD65] 
Corn Wave Ponytail Extension Clip in - 22 Inch Long Wavy Curly Wrap Around Pony Tail Heat Resistant Synthetic Hairpiece for Women (Natural Black #1B) 
$9.98 

Action: think[B08H5DCD65 is long clip-in hair extension which is natural looking and less than 20 dollars. I can check i

Action: think[The first two items are not what I want. The third item is not what I want, but it is the closest one.]
Observation: OK.

Action: click[B07GJTKYJQ]
Observation: 
[Back to Search] 
[< Prev] 
Louisville Vegan Jerky - 5 Flavor Variety Pack, Non-GMO Soy Protein, Gluten-Free (Black Pepper, Buffalo Dill, Pepperoni, Maple Bacon, and Carolina BBQ, 3 Ounces) 
Price: $42.99 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now] 

Action: think[The price is too high and it is not what I want.]
Observation: OK.

4 0.4375 0.25 0.0
-------------
-----------------
4
Action: reset
Observation: 
Webshop 
Instruction:  
find me a high speed dual style package with  12" power amplifier car subwoofer, and price lower than 200.00 dollars 
[Search] 

Action: search[high speed dual style package 12" power amplifier car subwoofer]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B01BGW8GOM] 
MTX TNP212D2 12" 1200 Watt 4 Ohm Dual Loaded Car Audio Subwoofer Package wi

Action: search[xx-large black workout sweatpants relaxed fit]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B0962X9S1W] 
Adogirl Womens Casual High Wasited Joggers Sweatpants Drawstring Workout Yoga Lounge Pants with Pockets 
$12.99 to $13.29 
[B09PNGTJGS] 
2022 Womens Yoga Pants with Pocket High Waisted Seamless Workout Tennis Skirt Pants Soft Stretch Trendy Solid Athletic Tights 
$8.99 to $14.99 
[B09P5Y4MFX] 
Mens Shorts Cargo Shorts for Men Trousers for Men Classic Relaxed Fit Cargo Pants Outdoor Short Pants 
$26.99 

Action: think[B0962X9S1W and 2022 Womens Yoga Pants with Pocket High Waisted Seamless Workout Tennis Skirt Pants Soft Stretch Trendy Solid Athletic Tights are black workout sweatpants less then 30 dollars. I can check B0962X9S1W first.]
Observation: OK.

Action: click[B0962X9S1W]
Observation: 
[Back to Search] 
[< Prev] 
color [arctic plum][black][blue]
size [small][medium][large][x-large][xx-large]
Adogirl Womens Casual High Wasited Joggers S

Action: click[B08B5XZ7LR]
Observation: 
[Back to Search] 
[< Prev] 
New AKB73775801 Remote for LG Blu-ray Disc Player Home Theater System BH5140 S54S1-S S54T1-W BH5440P S54T1-S S54T1-C BH5540T BH5140SF0 LHB655 S65T1-S S65T1-C S65T1-W LHB655FB LHB675 LHB675FB S65T3-S 
Price: $7.95 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now] 

Action: think[The item is a remote control for lg blu-ray dvd home theater system and seems good to buy.]
Observation: OK.

Action: click[Buy Now]
Observation: Your score (min 0.0, max 1.0): 1.0

13 0.685897435897436 0.46153846153846156 0.0
-------------
-----------------
13
Action: reset
Observation: 
Webshop 
Instruction:  
i need a long lasting 6.76 fl oz bottle of l'eau d'issey, and price lower than 100.00 dollars 
[Search] 

Action: search[long lasting 6.76 fl oz bottle l'eau d'issey]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B000VOHH8I] 
L'eau D'issey By Issey Miyake for MenEau De Toilette Spray, 6.7 Fl Oz Bottl

Action: think[For lead free luxury scented candle which last for 25+ hours, the item has options 'voyager' and '25+ hour' and seems good to buy.]
Observation: OK.

Action: click[voyager]
Observation: You have clicked voyager.

Action: click[25+ hour]
Observation: You have clicked 25+ hour.

Action: click[Buy Now]
Observation: Your score (min 0.0, max 1.0): 1.0

17 0.7401960784313726 0.5294117647058824 0.0
-------------
-----------------
17
Action: reset
Observation: 
Webshop 
Instruction:  
i am looking for mn4 color foundation for my sensitive skin, and price lower than 30.00 dollars 
[Search] 

Action: search[mn4 color foundation sensitive skin]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B077PR9TL4] 
Physicians Formula The Healthy Foundation with SPF 20, LW2, 1 Fl Oz 
$7.59 
[B07VLKYJNH] 
PÜR 4-in-1 Love Your Selfie Longwear Foundation & Concealer - Full Coverage Foundation & Concealer Makeup For Dark Spots, Blemishes and Imperfections - Long-Lasting Liqui

...

Action: think[B08331MC3W is super soft fleece throw for living room size 50"*40" and less than 30 dollars. I can check it.]
Observation: OK.

Action: click[B08331MC3W]
Observation: 
[Back to Search] 
[< Prev] 
color [1 photo][1 photo type2][1 photo type3][a-01][a-02][a-03][a-04][a-05][a-06][a-07][a-08][a-10][a-11][a-12][a-13][a-14][a-15][a-16][a-17][a-18][a-20][a-21][a-22][a-24][a-25][a-23]
size [30"x40"][40"x50"][50"x60"][51"x63"]
Lightweight Cozy Bed Blanket Throw Fuzzy Super Soft and Warm Throw Flannel Blankets for Couch Sofa Bed Beach Travel,Turquoise Tulip (30 x 40 Inches) 
Price: $14.99 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now] 

Action: think[For super soft fleece throw for living room size 50"*40", the item has options '30"x40"' and seems good to buy.]
Observation: OK.

Action: click[30"x40"]
Observation: You have clicked 30"x40".

Action: click[Buy Now]
Observation: Your score (min 0.0, max 1.0): 0.75

478 0.6328974895397488 0.3472803347280335 0.0
--------

Action: search[16.5 inch dining room chair pads]
Observation: 
[Back to Search] 
Page 1 (Total results: 50) 
[Next >] 
[B09PG8CYST] 
Modern Dining Chairs with Chrome Legs, Comfortable Kitchen Chairs Faux Leather Dining Room Chairs for Kitchen, Living, Bedroom, Office Side Chairs Set of 4 (White) 
$385.99 
[B08K8V3PWB] 
Modern Dining Chairs Set of 6 - Faux Leather Dining Room Chairs, Comfortable Kitchen Chairs with Chrome Legs for Kitchen, Living Room, Bedroom, Dining Room Side Chairs Set of 6 (Black) 
$597.97 
[B08K8V52F1] 
Modern Dining Chairs Set of 6 - Faux Leather Dining Room Chairs, Comfortable Kitchen Chairs with Chrome Legs for Kitchen, Living Room, Bedroom, Dining Room Side Chairs Set of 6 (White) 
$597.97 

Action: think[The first two items are not 16.5 inch dining room chair pads. The third item, 'Modern Dining Chairs Set of 6 - Faux Leather Dining Room Chairs, Comfortable Kitchen Chairs with Chrome Legs for Kitchen, Living Room, Bedroom, Dining Room Side Chairs Set of 6 (Whi

Action: think[B08PZC68W2 is deep brown clog with rubber sole for size 8 and the price is lower than 90 dollars.]
Observation: OK.

Action: click[B08PZC68W2]
Observation: 
[Back to Search] 
[< Prev] 
size [5][6][7][8][9][10][10.5][11]
color [black][brown][light brown][burned orange]
Sanita Ursana Open Clogs Original Handmade Leather Wooden Clogs for Women 
Price: $78.29 to $79.99 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now] 

Action: think[For deep brown clog with rubber sole for size 8, the item has options 'brown' and '8' and seems good to buy.]
Observation: OK.

Action: click[brown]
Observation: You have clicked brown.

Action: click[8]
Observation: You have clicked 8.

Action: click[Buy Now]
Observation: Your score (min 0.0, max 1.0): 1.0

487 0.6356091718001368 0.351129363449692 0.0
-------------
-----------------
487
Action: reset
Observation: 
Webshop 
Instruction:  
i am looking for a long handled body brush, and price lower than 70.00 dollars 
[Search] 

Actio

Action: think[B09SPZ8HC2 is 5 pack of fully cooked chicken breast strips less then 140 dollars. I can check it.]
Observation: OK.

Action: click[B09SPZ8HC2]
Observation: 
[Back to Search] 
[< Prev] 
size [4 - pack][5 - pack][6 - pack][7 - pack]
Tyson All natural Perfectly Crispy Chicken Breast Strips with Rib Meat - Fully Cooked - No Antibiotics Ever - 6 Pack (25 oz Each) - Ready Set Gourmet Donate a meal Program 
Price: $109.99 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now] 

Action: think[For 5 pack of fully cooked chicken breast strips, the item has option '5 - pack' and seems good to buy.]
Observation: OK.

Action: click[5 - pack]
Observation: You have clicked 5 - pack.

Action: click[Buy Now]
Observation: Your score (min 0.0, max 1.0): 1.0

492 0.636602303523035 0.35365853658536583 0.0
-------------
-----------------
492
Action: reset
Observation: 
Webshop 
Instruction:  
i want a king size twin bed with storage compartments 
[Search] 

Action: search[king size twi

Action: think[B091F3RWX8 is brown eco friendly nikahoo small storage baskets and it is less than 50 dollars.]
Observation: OK.

Action: click[B091F3RWX8]
Observation: 
[Back to Search] 
[< Prev] 
color [01 brown-3pcs][04 black-3pcs][06 brown-2pcs][07 ginger yellow-2pcs][09 black-2pcs]
NIKAHOO Small Basket Bin Leather Storage Basket, Foldable Shelf Desk Organizer Trays for Living Room, Office, Nursery, Home Décor 
Price: $15.99 
Rating: N.A. 
[Description] 
[Features] 
[Reviews] 
[Buy Now] 

Action: think[For brown eco friendly nikahoo small storage baskets, the item has options '01 brown-3pcs' and it seems good to buy.]
Observation: OK.

Action: click[01 brown-3pcs]
Observation: You have clicked 01 brown-3pcs.

Action: click[Buy Now]
Observation: Your score (min 0.0, max 1.0): 1.0

497 0.6370053655264921 0.3561368209255533 0.0
-------------
-----------------
497
Action: reset
Observation: 
Webshop 
Instruction:  
i am looking for ultra hd motion detection surveillance dome camera color